In [ ]:
%load_ext autoreload
import numpy as np
import pandas as pd
import heapq
import torch
from random import choice
import math
from torch import LongTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

def get_test_instances_with_random_samples(data, random_samples,num_items,device):
    user_input = np.zeros((random_samples+1))
    item_input = np.zeros((random_samples+1))

    # positive instance
    user_input[0] = data[0]
    item_input[0] = data[1]
    i = 1
    # negative instances
    checkList = data[1]
    for t in range(random_samples):
        j = np.random.randint(num_items)
        while j == checkList:
            j = np.random.randint(num_items)
        user_input[i] = data[0]
        item_input[i] = j
        i += 1
    return torch.LongTensor(user_input).to(device), torch.LongTensor(item_input).to(device)

def get_instances_with_random_neg_samples(train,num_items, num_negatives,device):
    user_input = np.zeros((len(train)+len(train)*num_negatives))
    item_input = np.zeros((len(train)+len(train)*num_negatives))
    labels = np.zeros((len(train)+len(train)*num_negatives))

    neg_samples = choice(num_items, size=(10*len(train)*num_negatives,)) # multiply by 2 to make sure, we dont run out of negative samples
    neg_counter = 0
    i = 0
    for n in range(len(train)):
        # positive instance
        user_input[i] = train['user_id'][n]
        item_input[i] = train['like_id'][n]
        labels[i] = 1
        i += 1
        # negative instances
        checkList = list(train['like_id'][train['user_id']==train['user_id'][n]])
        for t in range(num_negatives):
            j = neg_samples[neg_counter]
            while j in checkList:
                neg_counter += 1
                j = neg_samples[neg_counter]
            user_input[i] = train['user_id'][n]
            item_input[i] = j
            labels[i] = 0
            i += 1
            neg_counter += 1
    return torch.LongTensor(user_input).to(device), torch.LongTensor(item_input).to(device), torch.FloatTensor(labels).to(device)

def evaluate_model(model,df_val,top_K,random_samples, num_items):
    model.eval()
    avg_HR = np.zeros((len(df_val),top_K))
    avg_NDCG = np.zeros((len(df_val),top_K))

    for i in range(len(df_val)):
        test_user_input, test_item_input = get_test_instances_with_random_samples(df_val[i], random_samples,num_items,device)
        y_hat = model(test_user_input, test_item_input)
        y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
        test_item_input = test_item_input.cpu().detach().numpy().reshape((-1,))
        map_item_score = {}
        for j in range(len(y_hat)):
            map_item_score[test_item_input[j]] = y_hat[j]
        for k in range(top_K):
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = test_item_input[0]
            avg_HR[i,k] = getHitRatio(ranklist, gtItem)
            avg_NDCG[i,k] = getNDCG(ranklist, gtItem)

    avg_HR = np.mean(avg_HR, axis = 0)
    avg_NDCG = np.mean(avg_NDCG, axis = 0)
    return avg_HR, avg_NDCG


test_data = pd.read_csv("evaluations/nfcf/test_userPages.csv")
test_data

In [ ]:
from evaluate import Evaluate
from data import DataGenerator
import torch.nn as nn
class neuralCollabFilter(nn.Module):
    def __init__(self, num_users, num_likes, embed_size, num_hidden, output_size):
        super(neuralCollabFilter, self).__init__()
        self.user_emb = nn.Embedding(num_users, embed_size)
        self.like_emb = nn.Embedding(num_likes, embed_size)
        self.fc1 = nn.Linear(embed_size * 2, num_hidden[0])
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(num_hidden[0], num_hidden[1])
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(num_hidden[1], num_hidden[2])
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(num_hidden[2], num_hidden[3])
        self.relu4 = nn.ReLU()
        self.outLayer = nn.Linear(num_hidden[3], output_size)
        self.out_act = nn.Sigmoid()

    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.like_emb(v)
        out = torch.cat([U, V], dim=1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.outLayer(out)
        out = self.out_act(out)
        return out


ncf = neuralCollabFilter(6040, 3416, 128, np.array([128, 64, 32, 16]), 1).to(device)
ncf.load_state_dict(torch.load("evaluations/nfcf/preTrained_NCF",  map_location=torch.device('cpu')))
ncf.to(device)

data = DataGenerator()
evaluator = Evaluate(ncf, data, device=device)

In [ ]:
from time import time
t1 = time()
avg_HR_preTrain, avg_NDCG_preTrain = evaluate_model(ncf,data.test[['uid', 'mid']].values,15,100, 3416)
t2 = time()

t2-t1

In [ ]:
avg_HR_preTrain


In [ ]:
t1 = time()
hr, ndcg = evaluator()
t2 = time()
t2 - t1

In [ ]:
hr

In [ ]:
from time import time
def rank(l, item):
    # rank of the test item in the list of negative instances
    # returns the number of elements that the test item is bigger than

    index = 0
    for element in l:
        if element > item:
            index += 1
            return index
        index += 1
    return index
def eval_model(model, data, num_users=6040):
    # Evaluates the model and returns HR@10 and NDCG@10
    hits = 0
    ndcg = 0
    for u in range(num_users):
        user = data.testing_tensors[0][u].squeeze().to(device)
        item = data.testing_tensors[1][u].squeeze().to(device)
        y = model(user, item)

        y = y.tolist()
        y = sum(y, [])
        first = y.pop(0)
        y.sort()
        ranking = rank(y, first)
        if ranking > 90:
            hits += 1
            ndcg += np.log(2) / np.log(len(user) - ranking + 1)

    hr = hits / num_users
    ndcg = ndcg / num_users
    return hr, ndcg

t1 = time()
hr2, ndcg2 = eval_model(ncf, data)
t2 = time()

print(t2-t1)
print(hr2)

In [ ]:
# from tf

import math
import heapq # for retrieval topK
import multiprocessing
import numpy as np
from time import time

# import scipy.sparse as sp

class Dataset(object):
    '''
    classdocs
    '''

    def __init__(self, path):
        '''
        Constructor
        '''
        # self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)

        # self.num_users, self.num_items = self.trainMatrix.shape

    @staticmethod
    def load_rating_file_as_list(filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList

    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1: ]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList

    # def load_rating_file_as_matrix(self, filename):
    #     '''
    #     Read .rating file and Return dok matrix.
    #     The first line of .rating file is: num_users\t num_items
    #     '''
    #     # Get number of users and items
    #     num_users, num_items = 0, 0
    #     with open(filename, "r") as f:
    #         line = f.readline()
    #         while line != None and line != "":
    #             arr = line.split("\t")
    #             u, i = int(arr[0]), int(arr[1])
    #             num_users = max(num_users, u)
    #             num_items = max(num_items, i)
    #             line = f.readline()
    #     # Construct matrix
    #     mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    #     with open(filename, "r") as f:
    #         line = f.readline()
    #         while line != None and line != "":
    #             arr = line.split("\t")
    #             user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
    #             if (rating > 0):
    #                 mat[user, item] = 1.0
    #             line = f.readline()
    #     return mat

dataset = Dataset('evaluations/tf/ml-1m')
testRatings, testNegatives = dataset.testRatings, dataset.testNegatives

# Global variables that are shared across processes
_model = ncf
_testRatings = testRatings
_testNegatives = testNegatives
_K = 10

def tf_eval(num_thread):
    """
    Evaluate the performance (Hit_Ratio, NDCG) of top-K recommendation
    Return: score of each test rating.
    """
    # _model = model
    # _testRatings = testRatings
    # _testNegatives = testNegatives
    # _K = K

    hits, ndcgs = [],[]
    if num_thread > 1: # Multi-thread
        pool = multiprocessing.Pool(processes=num_thread)
        res = pool.map(eval_one_rating, range(len(_testRatings)))
        pool.close()
        pool.join()
        hits = [r[0] for r in res]
        ndcgs = [r[1] for r in res]
        return (hits, ndcgs)
    # Single thread
    for idx in range(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)
    return (hits, ndcgs)

def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    test_user_input, test_item_input = get_test_instances_with_random_samples(test_data.values[idx], 100,data.num_movies,device)
    predictions = _model(test_user_input, test_item_input)
    # predictions = _model(LongTensor(users), LongTensor(items))
    map_item_score = dict(zip(items, predictions))

    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [ ]:
t1 = time()
(hits, ndcgs) = tf_eval(1)
t2 = time()
t2 - t1

In [ ]:
print(f'original {avg_HR_preTrain}\n opt {hr2} \ntorch{np.array(hits).mean()}')


In [ ]:
from torch.utils.data import DataLoader

num_epochs = 25
batch_size = 2048
learning_rate = .001
top_k = 10

def train_ncf(model):
    # data.get_train_instances(seed=e)
    optimizer = torch.optim.AdamW(model.parameters(),  lr=learning_rate, weight_decay=1e-6)

    dataloader = DataLoader(data, batch_size=batch_size,
                            shuffle=True, num_workers=0)
    t1 = time()

    it_per_epoch = len(data) / batch_size
    for i in range(num_epochs):
        model.train()
        print("Starting epoch ", i + 1)
        j = 0
        for batch in dataloader:
            u, m, r = batch
            # move tensors to cuda
            u = u.to(device)
            m = m.to(device)
            r = r.to(device)

            y_hat = model(u.squeeze(1), m.squeeze(1))

            loss = torch.nn.BCELoss()  # (weight=w, reduction="mean")
            loss = loss(y_hat, r.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if j % int(1 + it_per_epoch / 10) == 0:
                print("Progress: ", round(100 * j / it_per_epoch), "%")
            j+=1

        # Epoch metrics
        t2 = time()
        print("Epoch time:", round(t2 - t1), "seconds")
        print("Loss:", loss / i)
        ncf.eval()


        print('baseline ')
        t1 = time()
        hr, ndcg = evaluate_model(ncf,data.test[['uid', 'mid']].values,10,100, 3416)
        t2 = time()
        print("Evaluation time:", round(t2 - t1), "seconds")
        print(f"HR@{top_k}:{hr[-1]}")

        print("(evaluator)...")
        t1 = time()
        hr, ndcg = evaluator()
        # hr, ndcg = evaluate(model, data.test, top_K)
        t2 = time()
        print("Evaluation time:", round(t2 - t1), "seconds")
        print(f"HR@{top_k}:{hr}")

        print("Evaluating (eval_model)...")
        t1 = time()
        hr, ndcg = eval_model(model, data)
        t2 = time()
        print("Evaluation time:", round(t2 - t1), "seconds")
        print(f"HR@{top_k}:{hr}")


        print("Evaluating (tf)...")
        t1 = time()
        hr, ndcg = tf_eval(1)
        t2 = time()

        print("Evaluation time:", round(t2 - t1), "seconds")
        print(f"HR@{top_k}:{np.array(hr).mean()}")
        # new
        # HR, NDCG = evaluate_model(model, data, validation=False)
        # updated
        # hr, ndcg = evaluate_model(model, data.test, top_K, random_samples)
        # original
        loss = 0
        print()

    print("Done")

In [ ]:
train_ncf(ncf)